In [1]:
import warnings
import os
from IPython.display import Audio, display
import numpy as np
import librosa
from librosa.display import specshow
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct, ifft
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from scipy.special import logsumexp
from collections import defaultdict, namedtuple
import random

import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torchvision.datasets.folder import default_loader
from torch.utils.data import DataLoader

from tqdm import tqdm

from tensorboardX import SummaryWriter

from hyperparameters import *

# warnings.simplefilter('error')
%matplotlib inline


np.random.seed(123)
random.seed(123)

In [8]:
def extract_mfcc(sound, sampling_rate=SAMPLE_RATE, shift=32., L=128., mel_coefs=120, mfcc_coefs=12, alpha=0.9, eps=1e-9):
    mfcc = librosa.feature.mfcc(y=sound, sr=sampling_rate, n_mfcc=mfcc_coefs)
    energy = librosa.feature.rms(sound)
    mfcc_energy = np.vstack((mfcc, energy))
    dx = librosa.feature.delta(mfcc_energy, order=1, width=3)
    d2x = librosa.feature.delta(mfcc_energy, order=2, width=3)
    res_features = np.vstack((mfcc_energy, dx, d2x)).T
    return res_features.astype(np.float32)


def show(audio):
    return Audio(audio, rate=SAMPLE_RATE)

In [9]:
def get_data(data_path=DATA_PATH):
    train_data = {}
    test_data = {}
    for uid, user_id in tqdm(enumerate(os.listdir(data_path))):
        videos = os.listdir(os.path.join(data_path, user_id))
        random.shuffle(videos)
        split = round(TRAIN_SPLIT * len(videos))
        train_data[uid] = []
        test_data[uid] = []
        for vid, video_id in enumerate(videos):
            for file in os.listdir(os.path.join(data_path, user_id, video_id)):
                full_path = os.path.join(data_path, user_id, video_id, file)
                record = librosa.load(full_path, sr=SAMPLE_RATE)[0]
                for i in range(0, len(record) - NUM_FRAMES + 1, NUM_FRAMES): # last part with len < NUM_FRAMES is skipped
                    x = record[i:i+NUM_FRAMES]
                    if vid < split:
                        train_data[uid].append(extract_mfcc(x))
                    else:
                        test_data[uid].append(extract_mfcc(x))
    return train_data, test_data

In [10]:
class Records(Dataset):
    def __init__(self, records, pos=5, neg=5):
        self.indices = []
        self.target = []
        self.data = records
        for u, data in records.items():
            for i, x in enumerate(data):
                for _ in range(pos):
                    pos_x = i
                    while pos_x == i:
                        pos_x = random.randrange(0, len(data))
                    self.indices.append((u, i, u, pos_x))
                    self.target.append(1.0)
                for _ in range(neg):
                    neg_u = u
                    while neg_u == u or len(records[neg_u]) == 0:
                        neg_u = random.randrange(0, len(records))
                    neg_x = random.randrange(0, len(records[neg_u]))
                    self.indices.append((u, i, neg_u, neg_x))
                    self.target.append(0.0)
    
    def __getitem__(self, index):
        u, x, neg_u, neg_x = self.indices[index]
        return (self.data[u][x], self.data[neg_u][neg_x]), self.target[index]
    
    def __len__(self):
        return len(self.indices)

In [11]:
train_raw, test_raw = get_data()
train_data = Records(train_raw)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

test_data = Records(test_raw)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

0it [00:00, ?it/s]/home/tark/.local/share/apps/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
1it [00:06,  6.58s/it]


KeyboardInterrupt: 

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, input_size=INPUT_SIZE, num_layers=NUM_LAYERS, lstm_units=128):
        super(self.__class__, self).__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_units, num_layers=num_layers)
        self.sim = nn.Linear(in_features=1, out_features=1)
        
    def forward(self, x, y):
        # x, y -- tensors of size (batch, frames, 39)
        x_h, _= self.lstm(x.transpose(0, 1))
        y_h, _= self.lstm(y.transpose(0, 1))
        
        x_emb = x_h[-1]
        y_emb = y_h[-1]
        
        sim = self.sim(nn.functional.cosine_similarity(x_emb, y_emb).unsqueeze(1)).squeeze(1)
        return sim

In [ ]:
def train(model, optimizer, criterion, train_loader, test_loader, device, n_epochs=N_EPOCHS, log_dir='./logs'):
    writer = SummaryWriter(log_dir)
    train_losses = []
    test_losses = []
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        k = 0
        
        for i, (records, labels) in enumerate(train_loader):
            records_a, records_b = records
            records_a = records_a.to(device)
            records_b = records_b.to(device)
            labels = labels.to(device).float()
            
            outputs = model(records_a, records_b)
            loss = criterion(outputs, labels)
            train_loss += loss.item()
            k += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss /= k
        
        model.eval()
        test_loss = 0
        k = 0
        for i, (records, labels) in enumerate(test_loader):
            records_a, records_b = records
            records_a = records_a.to(device)
            records_b = records_b.to(device)
            labels = labels.to(device).float()
            
            with torch.no_grad():
                outputs = model(records_a, records_b)
                loss = criterion(outputs, labels)
            test_loss += loss.item()
            k += 1
        test_loss /= k
        
        writer.add_scalar('simple/train', train_loss, epoch)
        writer.add_scalar('simple/test', test_loss, epoch)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
%time train(model, optimizer, criterion, train_loader, test_loader, device, n_epochs=1)